In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
import seaborn as sns
%matplotlib notebook

In [2]:
data = pd.read_csv('winemag-data-130k-v2.csv')
data = data.drop(['Unnamed: 0', 'designation', 'province', 'region_1', 'region_2', 'taster_twitter_handle'], axis=1)
data

,country,description,points,price,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,90,28.0,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,90,75.0,Paul Gregutt,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,90,30.0,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",90,32.0,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [3]:
data = data[data.country != 'nan']
data = data[data.description != 'nan']
data = data[data.points != 'nan']
data = data[data.price != 'nan']
data = data[data['taster_name'] != 'nan']
data = data[data.title != 'nan']
data = data[data.variety != 'nan']
data = data[data.winery != 'nan']
data

/opt/conda/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,country,description,points,price,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,90,28.0,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,90,75.0,Paul Gregutt,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,90,30.0,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",90,32.0,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [4]:
country = data['country'].unique()

In [5]:
avg = []
for i in country:
    avg.append(data[data.country == i]['points'].mean())

In [6]:
avg_points = {'Country':country, 'Average Points': avg} 

avg_points = pd.DataFrame(avg_points) 
 
avg_points = avg_points.sort_values(by=['Average Points'], ascending=False)
avg_points

,Country,Average Points
26,England,91.581081
33,India,90.222222
9,Austria,90.101345
5,Germany,89.851732
17,Canada,89.369650
13,Hungary,89.191781
42,China,89.000000
4,France,88.845109
22,Luxembourg,88.666667
8,Australia,88.580507


In [7]:
winery = data['winery'].unique()
winery

array(['Nicosia', 'Quinta dos Avidagos', 'Rainstorm', ...,
       'Mas de Pampelonne', 'Bodegas Eidosela', 'Penedo Borges'],
      dtype=object)

In [8]:
avg1 = []
for i in winery:
    avg1.append(data[data.winery == i]['points'].mean())

In [9]:
wavg_points = {'Winery':winery, 'Average Points': avg1} 

wavg_points = pd.DataFrame(wavg_points) 
 
wavg_points = wavg_points.sort_values(by=['Average Points'], ascending=False)
wavg_points

,Winery,Average Points
11466,Araujo,98.0
16486,Gandona,97.0
16485,Ovid,97.0
15600,J.L. Chave,97.0
7914,Salon,96.8
...,...,...
16310,Gran Familia,80.0
7130,Villa Viña,80.0
16334,Capay Valley,80.0
14868,Moss Roxx,80.0


In [10]:
grapes = data['variety'].unique()
grapes

array(['White Blend', 'Portuguese Red', 'Pinot Gris', 'Riesling',
       'Pinot Noir', 'Tempranillo-Merlot', 'Frappato', 'Gewürztraminer',
       'Cabernet Sauvignon', 'Nerello Mascalese', 'Chardonnay', 'Malbec',
       'Tempranillo Blend', 'Meritage', 'Red Blend', 'Merlot',
       "Nero d'Avola", 'Chenin Blanc', 'Gamay', 'Sauvignon Blanc',
       'Viognier-Chardonnay', 'Primitivo', 'Catarratto', 'Inzolia',
       'Petit Verdot', 'Monica', 'Bordeaux-style White Blend', 'Grillo',
       'Sangiovese', 'Cabernet Franc', 'Champagne Blend',
       'Bordeaux-style Red Blend', 'Aglianico', 'Petite Sirah',
       'Touriga Nacional', 'Carmenère', 'Albariño', 'Petit Manseng',
       'Rosé', 'Zinfandel', 'Vernaccia', 'Rosato', 'Grüner Veltliner',
       'Viognier', 'Vermentino', 'Grenache Blanc', 'Syrah', 'Nebbiolo',
       'Shiraz-Cabernet Sauvignon', 'Pinot Blanc', 'Alsace white blend',
       'Barbera', 'Rhône-style Red Blend', 'Portuguese White', 'Graciano',
       'Tannat-Cabernet', 'Sauvign

In [11]:
avg = []
for i in grapes:
    avg.append(data[data.variety == i]['points'].mean())

In [12]:
gavg_points = {'Grapes':grapes, 'Average Points': avg} 

gavg_points = pd.DataFrame(gavg_points) 
 
gavg_points = gavg_points.sort_values(by=['Average Points'], ascending=False)
gavg_points

,Grapes,Average Points
399,Tinta del Pais,95.000000
486,Terrantez,95.000000
633,Gelber Traminer,95.000000
487,Bual,94.142857
558,Riesling-Chardonnay,94.000000
...,...,...
671,Picapoll,82.000000
643,Shiraz-Tempranillo,82.000000
536,Airen,81.666667
666,Chancellor,80.500000


In [13]:
wine = data['title'].unique()
wine

array(['Nicosia 2013 Vulkà Bianco  (Etna)',
       'Quinta dos Avidagos 2011 Avidagos Red (Douro)',
       'Rainstorm 2013 Pinot Gris (Willamette Valley)', ...,
       'Domaine Gresser 2013 Kritt Gewurztraminer (Alsace)',
       'Domaine Marcel Deiss 2012 Pinot Gris (Alsace)',
       'Domaine Schoffit 2012 Lieu-dit Harth Cuvée Caroline Gewurztraminer (Alsace)'],
      dtype=object)

In [14]:
avg = []
for i in wine:
    avg.append(data[data.title == i]['points'].mean())
avg

[87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 85.0,
 85.0,
 85.0,
 85.0,
 85.0,
 85.0,
 85.0,
 85.0,
 86.0,
 86.0,
 86.0,
 86.0,
 85.5,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 86.0,
 88.0,
 87.5,
 88.0,
 88.0,
 88.0,
 88.0,
 88.0,
 88.0,
 88.0,
 88.0,
 88.0,
 88.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 87.0,
 92.0,
 92.0,
 92.0,
 92.0,
 92.0,
 92.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 91.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,

In [15]:
wine_avg_points = {'Wine':wine, 'Average Points': avg} 

wine_avg_points = pd.DataFrame(wine_avg_points) 
 
wine_avg_points = wine_avg_points.sort_values(by=['Average Points'], ascending=False)
wine_avg_points[wine_avg_points['Average Points'] == 100]

,Wine,Average Points
84249,Salon 2006 Le Mesnil Blanc de Blancs Brut Char...,100.0
103443,Château Léoville Las Cases 2010 Saint-Julien,100.0
41019,Casa Ferreirinha 2008 Barca-Velha Red (Douro),100.0
56012,Château Léoville Barton 2010 Saint-Julien,100.0
113445,Cayuse 2008 Bionic Frog Syrah (Walla Walla Val...,100.0
44375,Cardinale 2006 Cabernet Sauvignon (Napa Valley),100.0
105287,Charles Smith 2006 Royal City Syrah (Columbia ...,100.0
44358,Biondi Santi 2010 Riserva (Brunello di Montal...,100.0
103440,Château Lafite Rothschild 2010 Pauillac,100.0
103441,Casanova di Neri 2007 Cerretalto (Brunello di...,100.0


In [16]:
taster = data['taster_name'].unique()
taster

array(['Kerin O’Keefe', 'Roger Voss', 'Paul Gregutt',
       'Alexander Peartree', 'Michael Schachner', 'Anna Lee C. Iijima',
       'Virginie Boone', 'Matt Kettmann', nan, 'Sean P. Sullivan',
       'Jim Gordon', 'Joe Czerwinski', 'Anne Krebiehl\xa0MW',
       'Lauren Buzzeo', 'Mike DeSimone', 'Jeff Jenssen',
       'Susan Kostrzewa', 'Carrie Dykes', 'Fiona Adams',
       'Christina Pickard'], dtype=object)

In [17]:
max_point = []
for i in taster:
    mp = data[data.taster_name == i]['points'].max()
    max_point.append(mp)
max_point


[100,
 100,
 100,
 91,
 98,
 98,
 99,
 97,
 nan,
 97,
 97,
 100,
 97,
 95,
 94,
 97,
 94,
 92,
 91,
 93]

In [18]:
taster_fav = {'Taster name':taster, 'Point': max_point} 
  
# Create DataFrame 
taster_fav = pd.DataFrame(taster_fav) 
  
# Print the output. 
taster_fav = taster_fav.sort_values(by=['Point'], ascending=False)
taster_fav

,Taster name,Point
0,Kerin O’Keefe,100.0
1,Roger Voss,100.0
2,Paul Gregutt,100.0
11,Joe Czerwinski,100.0
6,Virginie Boone,99.0
4,Michael Schachner,98.0
5,Anna Lee C. Iijima,98.0
15,Jeff Jenssen,97.0
12,Anne Krebiehl MW,97.0
10,Jim Gordon,97.0
